# XLNetExtAbsSum for Abstractive Text Summarization
Brandon Castaing and Aditi Das - 12/1/2019

## Preliminary Setup

In [21]:
# Install libraries
#!pip3 install -q tensorflow tensorflow-datasets tf-nightly matplotlib pyyaml h5py 
#nltk pandas sentencepiece transformers torch torchvision --user

In [22]:
# Import Libraries
from __future__ import absolute_import, division, print_function, unicode_literals
import os, sys, re, json, time, datetime, shutil
import nltk
import sentencepiece as sp
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from transformers import *
import torch

In [23]:
# Check versions
print(np.__version__)
print(pd.__version__)
print(tf.__version__)
print(torch.__version__)

1.17.3
0.25.2
2.0.0
1.3.0


## Data Loading, Inspection, and EDA

In [24]:
dataset, info = tfds.load("cnn_dailymail", data_dir="./tf_cnndailymail_dataset", with_info=True)

INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset cnn_dailymail (./tf_cnndailymail_dataset/cnn_dailymail/plain_text/0.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from ./tf_cnndailymail_dataset/cnn_dailymail/plain_text/0.0.2


In [25]:
train = dataset['train']
validation = dataset['validation']
test = dataset['test']

In [26]:
#print(info)

# Build your input pipeline
train = train.shuffle(1000).batch(128).prefetch(10)
print(train)
print(train.take(1))
#for row in train.take(1):
    #print(f"First: {row['article']}")
    #print(f"Second: {row['highlights']}")

<PrefetchDataset shapes: {article: (None,), highlights: (None,)}, types: {article: tf.string, highlights: tf.string}>
<TakeDataset shapes: {article: (None,), highlights: (None,)}, types: {article: tf.string, highlights: tf.string}>


## Load XLNet and BertExtAbsSum

In [38]:
# Old attempts to load XLNet
model = tf.keras.Model()
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore('./xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt')

#processor = sp.SentencePieceProcessor()
#processor.Load("./xlnet_cased_L-24_H-1024_A-16/spiece.model")
#print(processor)

#model = tf.saved_model.load("./xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt")

print(checkpoint)
print(model)
print(type(model))
#print(model.summary(15))

<class 'tensorflow.python.keras.engine.training.Model'>


In [37]:
# Transformers library configurations
model_class, tokenizer_class, pretrained_weights = (TFXLNetModel, XLNetTokenizer, 'xlnet-base-cased')
print(f"Model Class: {model_class}")
print(f"Tokenizer Class: {tokenizer_class}")
print(f"Pretrained Weights: {pretrained_weights}")

# Load XLNets word embeddings (a.k.a. encoder)
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
print(f"Tokenizer: {tokenizer}")

# Load XLNet model
model = model_class.from_pretrained(pretrained_weights)
print(f"Model: {model}")

# Simple test
raw_text = "Here is some encoded text using XLNet!"
encoded = tokenizer.encode(raw_text, add_special_tokens=False)
decoded = tokenizer.decode(encoded)
print(f"Raw Text: {raw_text}\nXLNet Encoded Text: {encoded}\nXLNet Decoded Text: {decoded}")

#Input_ids = torch.tensor([tokenizer.encode(raw_text, add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
# Ensures model weights are not modified
#with torch.no_grad():
#    last_hidden_states = model(input_ids)[0]

Model Class: <class 'transformers.modeling_tf_xlnet.TFXLNetModel'>
Tokenizer Class: <class 'transformers.tokenization_xlnet.XLNetTokenizer'>
Pretrained Weights: xlnet-base-cased
Tokenizer: <transformers.tokenization_xlnet.XLNetTokenizer object at 0x7f64e4b22470>
Model: <transformers.modeling_tf_xlnet.TFXLNetModel object at 0x7f64e4c35748>
Raw Text: Here is some encoded text using XLNet!
XLNet Encoded Text: [1960, 27, 106, 23147, 1758, 381, 17, 20545, 10395, 136]
XLNet Decoded Text: Here is some encoded text using XLNet!


## Dry run CNN/Daily Mail through XLNet and BertExtAbsSum